### Applied Data Science Capsone

##### Final Assignment

This note book will be mainly used for the capstone project.

In [45]:
import pandas as pd
import numpy as np
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

##### import block data and transform it to a pandas dataframe

In [313]:
block = pd.read_csv('~/Desktop/Python/KT.csv')
df = pd.DataFrame(block)
df

,FID,lat,lng,Count
0,0,22.272960,114.122259,1
1,1,22.273106,114.121010,1
2,2,22.273676,114.119808,1
3,3,22.274046,114.118835,1
4,4,22.274937,114.127660,1
5,5,22.275559,114.128685,1
6,6,22.277690,114.129876,1
7,7,22.279789,114.127924,1
8,8,22.280980,114.125666,1
9,9,22.277140,114.122845,1


##### Use geopy library to get the latitude and longitude values of Kennedy Town

In [199]:
address = 'Kennedy Town, Hong Kong'
geolocator = Nominatim(user_agent='foursquare_agent')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

22.28131165 114.12916039816602


##### Create a map of Hong Kong with blocks superimposed on top

In [280]:
map_KT_ = folium.Map(location=[latitude, longitude],zoom_start=15)
map_KT_

##### The blocks in Hong Kong

In [208]:
KT_geo = r'KT_GeoJSON.json'
map_KT = folium.Map(location=[latitude, longitude],zoom_start=15,tiles='cartodbpositron')
for lat, lng in zip(df['lat'],df['lng']):
    folium.CircleMarker(
        [lat,lng],
        radius=1,
        color='darkblue',
        fill=True,
        fill_color='darkblue').add_to(map_KT)
map_KT.choropleth(
    geo_data = KT_geo,
    data = df,
    columns=['FID','Count'],
    key_on='feature.properties.FID',
    fill_color = 'YlGnBu',
    fill_opacity = 0.5,
    line_color = 'white',
    line_opacity = 0.3)
map_KT

##### Define Foursquare Credentials and Version

In [218]:
credentials = pd.read_csv('~\Desktop\python\credentials.csv')
CLIENT_ID = credentials['CLIENT_ID'][0] # my Foursquare ID
CLIENT_SECRET = credentials['CLIENT_SECRET'][0] # my Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
print(CLIENT_ID)
print(CLIENT_SECRET)

1ISYE4G22BNNXDMIUOD3PLXPCLE0MVDY5OYGXO525V2I2GWP
IMOMN4L1EV50TTVCEWS35KFGWJOP0VV0E2EV41LVR033I2KZ


##### Explore the blocks in Hong Kong Island
Get the top 100 venues for each block within a radius of 500 meters.

In [227]:
#create a function to loop all the blocks
LIMIT=100
def getNearbyVenues(blocks, lats, lngs, radius=500):
    venues_list=[]
    for block, lat, lng in zip(blocks, lats, lngs):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        results = requests.get(url).json()["response"]["groups"][0]['items']
        venues_list.append([(block, lat, lng, v['venue']['name'], v['venue']['location']['lat'],v['venue']['location']['lng'], v['venue']['categories'][0]['name'])for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['BlockID','BlockLat', 'BlockLng', 'Venue','V_lat', 'V_lng', 'V_category']
    return(nearby_venues)

In [228]:
KT_venues = getNearbyVenues(blocks = df['FID'], lats=df['lat'],lngs=df['lng'])
print(KT_venues.shape)
KT_venues.head()

(2782, 7)


,BlockID,BlockLat,BlockLng,Venue,V_lat,V_lng,V_category
0,0,22.272960,114.122259,Sandy Bay 沙灣,22.270556,114.121667,Harbor / Marina
1,0,22.272960,114.122259,Mount Davis Battery (摩星嶺砲台),22.276731,114.122383,Historic Site
2,0,22.272960,114.122259,Sunset Pavilion (日落觀瀾亭),22.274402,114.118258,Harbor / Marina
3,0,22.272960,114.122259,Jockey Club Mount Davis Youth Hostel (賽馬會摩星嶺青年旅舍),22.277222,114.122279,Hostel
4,1,22.273106,114.121010,Sandy Bay 沙灣,22.270556,114.121667,Harbor / Marina


In [229]:
#check how many venues were returned for each neighborhood
KT_venues.groupby('BlockID').count()

,BlockLat,BlockLng,Venue,V_lat,V_lng,V_category
BlockID,,,,,,
0,4,4,4,4,4,4
1,4,4,4,4,4,4
2,4,4,4,4,4,4
3,4,4,4,4,4,4
4,2,2,2,2,2,2
5,4,4,4,4,4,4
6,16,16,16,16,16,16
7,71,71,71,71,71,71
8,67,67,67,67,67,67


In [230]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [232]:
#find how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(KT_venues['V_category'].unique())))

There are 77 uniques categories.


In [234]:
#analyze each block
# one hot encoding
KT_onehot = pd.get_dummies(KT_venues[['V_category']], prefix="", prefix_sep="")

# add BlockID column back to dataframe
KT_onehot['BlockID'] = KT_venues['BlockID'] 

# move BlockId column to the first column
fixed_columns = [KT_onehot.columns[-1]] + list(KT_onehot.columns[:-1])
KT_onehot = KT_onehot[fixed_columns]

#group rows by blockID and by taking the mean of the frequency of occurrence of each category
KT_grouped = KT_onehot.groupby('BlockID').mean().reset_index()

print(KT_grouped.shape)
KT_grouped

(46, 78)


,BlockID,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Bakery,Beer Garden,Boat or Ferry,Bookstore,...,Sports Bar,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Thai Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,5,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000
6,6,0.000000,0.00,0.000000,0.062500,0.062500,0.000000,0.000000,0.000000,0.000000,...,0.062500,0.000000,0.062500,0.000000,0.000000,0.000000,0.062500,0.062500,0.000000,0.062500
7,7,0.014085,0.00,0.000000,0.014085,0.014085,0.014085,0.014085,0.000000,0.000000,...,0.014085,0.000000,0.028169,0.014085,0.014085,0.014085,0.014085,0.014085,0.000000,0.042254
8,8,0.014925,0.00,0.000000,0.014925,0.014925,0.014925,0.014925,0.000000,0.000000,...,0.014925,0.000000,0.014925,0.000000,0.014925,0.014925,0.014925,0.000000,0.000000,0.044776
9,10,0.014925,0.00,0.000000,0.014925,0.014925,0.029851,0.014925,0.000000,0.000000,...,0.000000,0.000000,0.014925,0.014925,0.014925,0.014925,0.014925,0.029851,0.000000,0.044776


In [244]:
num_top_venues = 5
KT_grouped['BlockID']=KT_grouped['BlockID'].apply(str)
for hood in KT_grouped['BlockID']:
    print("----"+hood+"----")
    temp = KT_grouped[KT_grouped['BlockID'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----0----
                      venue  freq
0           Harbor / Marina  0.50
1                    Hostel  0.25
2             Historic Site  0.25
3  Mediterranean Restaurant  0.00
4   New American Restaurant  0.00


----1----
                      venue  freq
0           Harbor / Marina  0.50
1                    Hostel  0.25
2             Historic Site  0.25
3  Mediterranean Restaurant  0.00
4   New American Restaurant  0.00


----2----
                      venue  freq
0           Harbor / Marina  0.50
1                    Hostel  0.25
2             Historic Site  0.25
3  Mediterranean Restaurant  0.00
4   New American Restaurant  0.00


----3----
                      venue  freq
0           Harbor / Marina  0.50
1                    Hostel  0.25
2             Historic Site  0.25
3  Mediterranean Restaurant  0.00
4   New American Restaurant  0.00


----4----
                      venue  freq
0                Playground   0.5
1                  Mountain   0.5
2       American Restaur

In [245]:
#function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [266]:
#create a new dataframe and display the top 10 venues for each block
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['BlockID']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
blocks_venues_sorted = pd.DataFrame(columns=columns)
blocks_venues_sorted['BlockID'] = KT_grouped['BlockID']

for ind in np.arange(KT_grouped.shape[0]):
    blocks_venues_sorted.iloc[ind, 1:] = return_most_common_venues(KT_grouped.iloc[ind, :], num_top_venues)

blocks_venues_sorted


,BlockID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Harbor / Marina,Hostel,Historic Site,Fish & Chips Shop,Dessert Shop,Dim Sum Restaurant,Discount Store,Dumpling Restaurant,English Restaurant,Fast Food Restaurant
1,1,Harbor / Marina,Hostel,Historic Site,Fish & Chips Shop,Dessert Shop,Dim Sum Restaurant,Discount Store,Dumpling Restaurant,English Restaurant,Fast Food Restaurant
2,2,Harbor / Marina,Hostel,Historic Site,Fish & Chips Shop,Dessert Shop,Dim Sum Restaurant,Discount Store,Dumpling Restaurant,English Restaurant,Fast Food Restaurant
3,3,Harbor / Marina,Hostel,Historic Site,Fish & Chips Shop,Dessert Shop,Dim Sum Restaurant,Discount Store,Dumpling Restaurant,English Restaurant,Fast Food Restaurant
4,4,Mountain,Playground,Vietnamese Restaurant,Chinese Restaurant,Dessert Shop,Dim Sum Restaurant,Discount Store,Dumpling Restaurant,English Restaurant,Fast Food Restaurant
5,5,Mountain,Trail,Playground,Scenic Lookout,Fast Food Restaurant,Coffee Shop,Dessert Shop,Dim Sum Restaurant,Discount Store,Dumpling Restaurant
6,6,Chinese Restaurant,Vietnamese Restaurant,Sports Bar,Mountain,Market,Japanese Restaurant,Playground,French Restaurant,Coffee Shop,Australian Restaurant
7,7,Japanese Restaurant,Coffee Shop,Chinese Restaurant,Café,Mexican Restaurant,Vietnamese Restaurant,Italian Restaurant,Fish & Chips Shop,Hong Kong Restaurant,Cha Chaan Teng
8,8,Japanese Restaurant,Coffee Shop,Chinese Restaurant,Café,Mexican Restaurant,Vietnamese Restaurant,Fish & Chips Shop,French Restaurant,Italian Restaurant,Cha Chaan Teng
9,10,Japanese Restaurant,Chinese Restaurant,Café,Coffee Shop,Vietnamese Restaurant,Bakery,Hong Kong Restaurant,Cha Chaan Teng,Mexican Restaurant,Trail


In [306]:
#cluster blocks
#run K-means to cluster the blocks into 5 clusters
kclusters = 3

KT_grouped_clustering = KT_grouped.drop('BlockID', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(KT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 2, 2, 0, 0, 0, 0])

In [314]:
#add clustering labels
#blocks_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
blocks_venues_sorted['Cluster Labels']=kmeans.labels_
df['FID']=df['FID'].apply(str)
a = df
b = blocks_venues_sorted
KT_merge = pd.merge(a, b, left_on='FID',right_on='BlockID')
# merge KT_grouped with df to add latitude/longitude for each block
#KT_merged = KT_merged.join(blocks_venues_sorted.set_index('BlockID'), on='FID')
KT_merge.drop(['FID','Count'],axis=1,inplace=True)
KT_merge.head() # check the last columns!

,lat,lng,BlockID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,22.272960,114.122259,0,Harbor / Marina,Hostel,Historic Site,Fish & Chips Shop,Dessert Shop,Dim Sum Restaurant,Discount Store,Dumpling Restaurant,English Restaurant,Fast Food Restaurant,1
1,22.273106,114.121010,1,Harbor / Marina,Hostel,Historic Site,Fish & Chips Shop,Dessert Shop,Dim Sum Restaurant,Discount Store,Dumpling Restaurant,English Restaurant,Fast Food Restaurant,1
2,22.273676,114.119808,2,Harbor / Marina,Hostel,Historic Site,Fish & Chips Shop,Dessert Shop,Dim Sum Restaurant,Discount Store,Dumpling Restaurant,English Restaurant,Fast Food Restaurant,1
3,22.274046,114.118835,3,Harbor / Marina,Hostel,Historic Site,Fish & Chips Shop,Dessert Shop,Dim Sum Restaurant,Discount Store,Dumpling Restaurant,English Restaurant,Fast Food Restaurant,1
4,22.274937,114.127660,4,Mountain,Playground,Vietnamese Restaurant,Chinese Restaurant,Dessert Shop,Dim Sum Restaurant,Discount Store,Dumpling Restaurant,English Restaurant,Fast Food Restaurant,2


In [309]:
#visualize the result
KT_merge['Cluster Labels']=KT_merge['Cluster Labels'].apply(float)
KT_merge['BlockID']=KT_merge['BlockID'].apply(int)
map_KT_C = folium.Map(location=[latitude, longitude],zoom_start=15,tiles='cartodbpositron')
map_KT_C.choropleth(
    geo_data = KT_geo,
    data = KT_merge,
    columns=['BlockID','Cluster Labels'],
    key_on='feature.properties.FID',
    fill_color = 'YlGnBu',
    fill_opacity = 0.5,
    line_opacity = 0.3)

map_KT_C

In [311]:
KT_merge.to_csv('~\Desktop\Python\Clustered_Blocks.csv')

#### Examine Clusters

##### Cluster 1

In [322]:
KT_merge.loc[KT_merge['Cluster Labels'] == 0.0, KT_merge.columns[list(range(2,KT_merge.shape[1]-1))]]

,BlockID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,6,Chinese Restaurant,Vietnamese Restaurant,Sports Bar,Mountain,Market,Japanese Restaurant,Playground,French Restaurant,Coffee Shop,Australian Restaurant
7,7,Japanese Restaurant,Coffee Shop,Chinese Restaurant,Café,Mexican Restaurant,Vietnamese Restaurant,Italian Restaurant,Fish & Chips Shop,Hong Kong Restaurant,Cha Chaan Teng
8,8,Japanese Restaurant,Coffee Shop,Chinese Restaurant,Café,Mexican Restaurant,Vietnamese Restaurant,Fish & Chips Shop,French Restaurant,Italian Restaurant,Cha Chaan Teng
9,10,Japanese Restaurant,Chinese Restaurant,Café,Coffee Shop,Vietnamese Restaurant,Bakery,Hong Kong Restaurant,Cha Chaan Teng,Mexican Restaurant,Trail
10,11,Japanese Restaurant,Coffee Shop,French Restaurant,Mexican Restaurant,Bakery,Italian Restaurant,Cha Chaan Teng,Ice Cream Shop,Hotpot Restaurant,Hotel
11,12,Japanese Restaurant,Coffee Shop,Chinese Restaurant,Vietnamese Restaurant,Café,Mexican Restaurant,Hong Kong Restaurant,French Restaurant,Fish & Chips Shop,Cha Chaan Teng
12,13,Japanese Restaurant,Chinese Restaurant,Coffee Shop,Vietnamese Restaurant,Café,Mexican Restaurant,Hong Kong Restaurant,Italian Restaurant,Bakery,French Restaurant
13,14,Japanese Restaurant,Chinese Restaurant,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Café,Italian Restaurant,Hong Kong Restaurant,French Restaurant,Fish & Chips Shop
14,15,Japanese Restaurant,Chinese Restaurant,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Café,Hong Kong Restaurant,Italian Restaurant,French Restaurant,Fish & Chips Shop
15,16,Japanese Restaurant,Chinese Restaurant,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Café,Hong Kong Restaurant,Italian Restaurant,French Restaurant,Fish & Chips Shop


##### Cluster 2

In [323]:
KT_merge.loc[KT_merge['Cluster Labels'] == 1.0, KT_merge.columns[list(range(2,KT_merge.shape[1]-1))]]

,BlockID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Harbor / Marina,Hostel,Historic Site,Fish & Chips Shop,Dessert Shop,Dim Sum Restaurant,Discount Store,Dumpling Restaurant,English Restaurant,Fast Food Restaurant
1,1,Harbor / Marina,Hostel,Historic Site,Fish & Chips Shop,Dessert Shop,Dim Sum Restaurant,Discount Store,Dumpling Restaurant,English Restaurant,Fast Food Restaurant
2,2,Harbor / Marina,Hostel,Historic Site,Fish & Chips Shop,Dessert Shop,Dim Sum Restaurant,Discount Store,Dumpling Restaurant,English Restaurant,Fast Food Restaurant
3,3,Harbor / Marina,Hostel,Historic Site,Fish & Chips Shop,Dessert Shop,Dim Sum Restaurant,Discount Store,Dumpling Restaurant,English Restaurant,Fast Food Restaurant
30,31,Harbor / Marina,Hostel,Playground,Fish & Chips Shop,Dessert Shop,Dim Sum Restaurant,Discount Store,Dumpling Restaurant,English Restaurant,Fast Food Restaurant


##### Cluster 3

In [324]:
KT_merge.loc[KT_merge['Cluster Labels'] == 2.0, KT_merge.columns[list(range(2,KT_merge.shape[1]-1))]]

,BlockID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,4,Mountain,Playground,Vietnamese Restaurant,Chinese Restaurant,Dessert Shop,Dim Sum Restaurant,Discount Store,Dumpling Restaurant,English Restaurant,Fast Food Restaurant
5,5,Mountain,Trail,Playground,Scenic Lookout,Fast Food Restaurant,Coffee Shop,Dessert Shop,Dim Sum Restaurant,Discount Store,Dumpling Restaurant
